In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from lightgbm import LGBMClassifier
import logging
import pickle


In [2]:
log_fmt = "[%(asctime)s] %(levelname)s in %(module)s: %(message)s"
logging.basicConfig(format=log_fmt, level=logging.INFO)

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
def extract_day(s):
    return s.apply(lambda x: int(x.split('-')[0][1:]))


def extract_hour(s):
    return s.apply(lambda x: int(x.split('-')[1][1:]))

In [5]:

# 加载训练、测试数据
with open('pkl/train.pkl', 'rb') as file:
    train = pickle.load(file)
logging.info("invite %s", train.shape)

with open('pkl/test.pkl', 'rb') as file:
    test = pickle.load(file)
logging.info("test %s", test.shape)

[2019-12-16 22:15:06,007] INFO in <ipython-input-5-3ff82ffe0811>: invite (9489162, 5)
[2019-12-16 22:15:06,427] INFO in <ipython-input-5-3ff82ffe0811>: test (1141683, 4)


In [6]:
# 加载问题
with open('pkl/question_info.pkl', 'rb') as file:
    ques = pickle.load(file)
    
del ques['title_t1'], ques['title_t2'], ques['desc_t1'], ques['desc_t2']    

logging.info("ques %s", ques.shape)

[2019-12-16 22:15:48,839] INFO in <ipython-input-6-e1f1016b64e4>: ques (1829900, 4)


In [7]:
# 加载回答
with open('pkl/answer_info.pkl', 'rb') as file:
    ans = pickle.load(file)

del ans['ans_t1'], ans['ans_t2']
logging.info("ans %s", ans.shape)

[2019-12-16 22:18:01,825] INFO in <ipython-input-7-d4da01d73f87>: ans (4513735, 19)


In [8]:
#将回答和问题信息按照qid进行合并
ans = pd.merge(ans, ques, on='qid')
del ques
#ans对于文本信息只留了topic
logging.info("ans %s", ans.shape)

[2019-12-16 22:18:09,245] INFO in <ipython-input-8-dad537edaf39>: ans (4513735, 22)


In [9]:
# 回答距提问的天数
ans['diff_qa_days'] = ans['a_day'] - ans['q_day']

# 时间窗口划分
# train
# val
train_start = 3838
train_end = 3867

val_start = 3868
val_end = 3874

label_end = 3867
label_start = label_end - 6

train_label_feature_end = label_end - 7
train_label_feature_start = train_label_feature_end - 22

train_ans_feature_end = label_end - 7
train_ans_feature_start = train_ans_feature_end - 50

val_label_feature_end = val_start - 1
val_label_feature_start = val_label_feature_end - 22

val_ans_feature_end = val_start - 1
val_ans_feature_start = val_ans_feature_end - 50

In [10]:
train_label_feature = train[(train['day'] >= train_label_feature_start) & (train['day'] <= train_label_feature_end)]
logging.info("train_label_feature %s", train_label_feature.shape)

val_label_feature = train[(train['day'] >= val_label_feature_start) & (train['day'] <= val_label_feature_end)]
logging.info("val_label_feature %s", val_label_feature.shape)

train_label = train[(train['day'] > train_label_feature_end)]

logging.info("train feature start %s end %s, label start %s end %s", train_label_feature['day'].min(),
             train_label_feature['day'].max(), train_label['day'].min(), train_label['day'].max())

logging.info("test feature start %s end %s, label start %s end %s", val_label_feature['day'].min(),
             val_label_feature['day'].max(), test['day'].min(), test['day'].max())

[2019-12-16 22:18:10,856] INFO in <ipython-input-10-db9ac0b937b4>: train_label_feature (6895493, 5)
[2019-12-16 22:18:11,275] INFO in <ipython-input-10-db9ac0b937b4>: val_label_feature (7583553, 5)
[2019-12-16 22:18:11,526] INFO in <ipython-input-10-db9ac0b937b4>: train feature start 3838 end 3860, label start 3861 end 3867
[2019-12-16 22:18:11,555] INFO in <ipython-input-10-db9ac0b937b4>: test feature start 3845 end 3867, label start 3868 end 3874


In [11]:
# 确定ans的时间范围
# 3807~3874
train_ans_feature = ans[(ans['a_day'] >= train_ans_feature_start) & (ans['a_day'] <= train_ans_feature_end)]

val_ans_feature = ans[(ans['a_day'] >= val_ans_feature_start) & (ans['a_day'] <= val_ans_feature_end)]

logging.info("train ans feature %s, start %s end %s", train_ans_feature.shape, train_ans_feature['a_day'].min(),
             train_ans_feature['a_day'].max())

logging.info("val ans feature %s, start %s end %s", val_ans_feature.shape, val_ans_feature['a_day'].min(),
             val_ans_feature['a_day'].max())

fea_cols = ['is_good', 'is_rec', 'is_dest', 'has_img', 'has_video', 'word_count',
            'reci_cheer', 'reci_uncheer', 'reci_comment', 'reci_mark', 'reci_tks',
            'reci_xxx', 'reci_no_help', 'reci_dis', 'diff_qa_days']

[2019-12-16 22:18:14,017] INFO in <ipython-input-11-aa15f72a2344>: train ans feature (3700178, 23), start 3810 end 3860
[2019-12-16 22:18:14,028] INFO in <ipython-input-11-aa15f72a2344>: val ans feature (3992334, 23), start 3817 end 3867


In [12]:
logging.info("train_label %s", train_label.shape)
logging.info("ans %s", ans.shape)

[2019-12-16 22:18:14,035] INFO in <ipython-input-12-34320fe37835>: train_label (2593669, 5)
[2019-12-16 22:18:14,036] INFO in <ipython-input-12-34320fe37835>: ans (4513735, 23)


In [13]:
def extract_feature1(target, label_feature, ans_feature):
    # 问题特征
    t1 = label_feature.groupby('qid')['label'].agg(['mean', 'sum', 'std', 'count']).reset_index()
    t1.columns = ['qid', 'q_inv_mean', 'q_inv_sum', 'q_inv_std', 'q_inv_count']
    target = pd.merge(target, t1, on='qid', how='left')

    # 用户特征
    t1 = label_feature.groupby('uid')['label'].agg(['mean', 'sum', 'std', 'count']).reset_index()
    t1.columns = ['uid', 'u_inv_mean', 'u_inv_sum', 'u_inv_std', 'u_inv_count']
    target = pd.merge(target, t1, on='uid', how='left')
    #
    # train_size = len(train)
    # data = pd.concat((train, test), sort=True)

    # 回答部分特征

    t1 = ans_feature.groupby('qid')['aid'].count().reset_index()
    t1.columns = ['qid', 'q_ans_count']
    target = pd.merge(target, t1, on='qid', how='left')

    t1 = ans_feature.groupby('uid')['aid'].count().reset_index()
    t1.columns = ['uid', 'u_ans_count']
    target = pd.merge(target, t1, on='uid', how='left')

    for col in fea_cols:
        t1 = ans_feature.groupby('uid')[col].agg(['sum', 'max', 'mean']).reset_index()
        t1.columns = ['uid', f'u_{col}_sum', f'u_{col}_max', f'u_{col}_mean']
        target = pd.merge(target, t1, on='uid', how='left')

        t1 = ans_feature.groupby('qid')[col].agg(['sum', 'max', 'mean']).reset_index()
        t1.columns = ['qid', f'q_{col}_sum', f'q_{col}_max', f'q_{col}_mean']
        target = pd.merge(target, t1, on='qid', how='left')
        logging.info("extract %s", col)
    return target

#共加了100个特征

In [ ]:
train_label = extract_feature1(train_label, train_label_feature, train_ans_feature)

[2019-12-16 22:18:52,340] INFO in <ipython-input-13-a62f06020d62>: extract is_good
[2019-12-16 22:19:04,199] INFO in <ipython-input-13-a62f06020d62>: extract is_rec
[2019-12-16 22:19:15,803] INFO in <ipython-input-13-a62f06020d62>: extract is_dest
[2019-12-16 22:19:27,974] INFO in <ipython-input-13-a62f06020d62>: extract has_img
[2019-12-16 22:19:40,110] INFO in <ipython-input-13-a62f06020d62>: extract has_video
[2019-12-16 22:19:52,499] INFO in <ipython-input-13-a62f06020d62>: extract word_count
[2019-12-16 22:20:05,067] INFO in <ipython-input-13-a62f06020d62>: extract reci_cheer


In [ ]:
pd.options.display.max_columns = None

In [ ]:
train_label.head()

In [ ]:
test = extract_feature1(test, val_label_feature, val_ans_feature)

In [ ]:
test.head()

In [ ]:
logging.info("train_label %s", train_label.shape)
logging.info("test %s", test.shape)
logging.info("ans %s", ans.shape)

In [ ]:
ans.head()

In [ ]:
#加载用户
with open('pkl/user.pkl', 'rb') as file:
    user = pickle.load(file)

del user['follow_topic'], user['inter_topic']
logging.info("user %s", user.shape)

In [ ]:
#删除用户特征中的常量
unq = user.nunique()
logging.info("user unq %s", unq)

for x in unq[unq == 1].index:
    del user[x]
    logging.info('del unq==1 %s', x)
logging.info("user %s", user.shape)

In [ ]:
#对多特征值进行编码
t = user.dtypes
cats = [x for x in t[t == 'object'].index if x not in ['follow_topic', 'inter_topic', 'uid']]
logging.info("user cat %s", cats)

for d in cats:
    lb = LabelEncoder()
    user[d] = lb.fit_transform(user[d])
    logging.info('encode %s', d)
    
logging.info("user %s", user.shape)

In [ ]:
#对uid和qid进行编码
q_lb = LabelEncoder()
q_lb.fit(list(train_label['qid'].astype(str).values) + list(test['qid'].astype(str).values))
train_label['qid_enc'] = q_lb.transform(train_label['qid'])
test['qid_enc'] = q_lb.transform(test['qid'])

u_lb = LabelEncoder()
u_lb.fit(user['uid'])
train_label['uid_enc'] = u_lb.transform(train_label['uid'])
test['uid_enc'] = u_lb.transform(test['uid'])
logging.info("user %s", user.shape)

In [ ]:
#merge user之前
logging.info("train_label %s", train_label.shape)
logging.info("test %s", test.shape)

In [ ]:
# merge user
train_label = pd.merge(train_label, user, on='uid', how='left')
test = pd.merge(test, user, on='uid', how='left')
logging.info("train shape %s, test shape %s", train_label.shape, test.shape)

data = pd.concat((train_label, test), axis=0, sort=True)
del train_label, test

In [27]:
logging.info("data %s", data.shape)

[2019-12-16 21:20:43,834] INFO in <ipython-input-27-906d24426bec>: data (3735352, 120)


In [28]:
# count编码
count_fea = ['uid_enc', 'qid_enc', 'gender', 'freq', 'uf_c1', 'uf_c2', 'uf_c3', 'uf_c4', 'uf_c5']
for feat in count_fea:
    col_name = '{}_count'.format(feat)
    data[col_name] = data[feat].map(data[feat].value_counts().astype(int))
    data.loc[data[col_name] < 2, feat] = -1
    data[feat] += 1
    data[col_name] = data[feat].map(data[feat].value_counts().astype(int))
    data[col_name] = (data[col_name] - data[col_name].min()) / (data[col_name].max() - data[col_name].min())

In [29]:
data.head()

,day,freq,gender,hour,label,q_ans_count,q_diff_qa_days_max,q_diff_qa_days_mean,q_diff_qa_days_sum,q_has_img_max,q_has_img_mean,q_has_img_sum,q_has_video_max,q_has_video_mean,q_has_video_sum,q_inv_count,q_inv_mean,q_inv_std,q_inv_sum,q_is_dest_max,q_is_dest_mean,q_is_dest_sum,q_is_good_max,q_is_good_mean,q_is_good_sum,q_is_rec_max,q_is_rec_mean,q_is_rec_sum,q_reci_cheer_max,q_reci_cheer_mean,q_reci_cheer_sum,q_reci_comment_max,q_reci_comment_mean,q_reci_comment_sum,q_reci_dis_max,q_reci_dis_mean,q_reci_dis_sum,q_reci_mark_max,q_reci_mark_mean,q_reci_mark_sum,q_reci_no_help_max,q_reci_no_help_mean,q_reci_no_help_sum,q_reci_tks_max,q_reci_tks_mean,q_reci_tks_sum,q_reci_uncheer_max,q_reci_uncheer_mean,q_reci_uncheer_sum,q_reci_xxx_max,q_reci_xxx_mean,q_reci_xxx_sum,q_word_count_max,q_word_count_mean,q_word_count_sum,qid,qid_enc,score,u_ans_count,u_diff_qa_days_max,u_diff_qa_days_mean,u_diff_qa_days_sum,u_has_img_max,u_has_img_mean,u_has_img_sum,u_has_video_max,u_has_video_mean,u_has_video_sum,u_inv_count,u_inv_mean,u_inv_std,u_inv_sum,u_is_dest_max,u_is_dest_mean,u_is_dest_sum,u_is_good_max,u_is_good_mean,u_is_good_sum,u_is_rec_max,u_is_rec_mean,u_is_rec_sum,u_reci_cheer_max,u_reci_cheer_mean,u_reci_cheer_sum,u_reci_comment_max,u_reci_comment_mean,u_reci_comment_sum,u_reci_dis_max,u_reci_dis_mean,u_reci_dis_sum,u_reci_mark_max,u_reci_mark_mean,u_reci_mark_sum,u_reci_no_help_max,u_reci_no_help_mean,u_reci_no_help_sum,u_reci_tks_max,u_reci_tks_mean,u_reci_tks_sum,u_reci_uncheer_max,u_reci_uncheer_mean,u_reci_uncheer_sum,u_reci_xxx_max,u_reci_xxx_mean,u_reci_xxx_sum,u_word_count_max,u_word_count_mean,u_word_count_sum,uf_b1,uf_b2,uf_b3,uf_b4,uf_b5,uf_c1,uf_c2,uf_c3,uf_c4,uf_c5,uid,uid_enc,uid_enc_count,qid_enc_count,gender_count,freq_count,uf_c1_count,uf_c2_count,uf_c3_count,uf_c4_count,uf_c5_count
0,3865,5,3,22,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Q2166419046,153995,297,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.000000,0.000000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,0,0,0,2114,191,262,928,2,M401693808,1508099,0.000004,0.000369,1.000000,0.961323,0.022204,0.371510,0.217985,0.062753,1.0
1,3862,5,3,15,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Q604029601,457970,415,2.0,2.0,1.000000,2.0,1.0,1.0,2.0,0.0,0.0,0.0,11.0,0.090909,0.301511,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.500,1.0,2.0,1.000000,2.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,139.0,137.000000,274.0,1,0,0,0,0,1520,230,1,507,2,M2317670257,657986,0.000004,0.000036,1.000000,0.961323,0.024503,0.111695,0.443770,0.002973,1.0
2,3867,2,3,4,0.0,32.0,13.0,9.531250,305.0,1.0,0.062500,2.0,0.0,0.0,0.0,152.0,0.375000,0.485723,57.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.375000,12.0,2.0,0.062500,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,136.0,42.125000,1348.0,Q2443223942,190474,296,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.000000,0.000000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0,0,552,227,189,816,2,M3544409350,1272354,0.000022,0.001341,1.000000,0.291847,0.006308,0.034012,0.400961,0.044143,1.0
3,3861,1,2,20,0.0,3.0,234.0,222.333333,667.0,1.0,0.333333,1.0,0.0,0.0,0.0,6.0,0.166667,0.408248,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1.333333,4.0,4.0,1.666667,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,332.0,117.333333,352.0,Q795459266,483104,380,6.0,1391.0,376.666667,2260.0,1.0,0.5,3.0,0.0,0.0,0.0,7.0,0.285714,0.4879

In [30]:
logging.info("data %s", data.shape)

[2019-12-16 21:21:34,699] INFO in <ipython-input-30-906d24426bec>: data (3735352, 129)


In [31]:
# 压缩数据
t = data.dtypes
for x in t[t == 'int64'].index:
    data[x] = data[x].astype('int32')

for x in t[t == 'float64'].index:
    data[x] = data[x].astype('float32')

data['wk'] = data['day'] % 7

In [32]:
feature_cols = [x for x in data.columns if x not in ('label', 'uid', 'qid', 'dt', 'day')]
# target编码
logging.info("feature size %s", len(feature_cols))

[2019-12-16 21:23:53,163] INFO in <ipython-input-32-efdad8952e8b>: feature size 126


In [33]:
logging.info("data %s", data.shape)

[2019-12-16 21:23:53,171] INFO in <ipython-input-33-906d24426bec>: data (3735352, 130)


In [34]:
#保存处理好的data
with open('pkl/data.pkl','wb') as file:
    pickle.dump(data,file)

In [ ]:
# 以下需要先运行完 特征工程文件

In [ ]:
######################################################################################################################################
#######################################特征处理#######################################################################################

In [49]:
import gc
import os
import time
import multiprocessing as mp
import logging
import pickle
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm, tqdm_notebook, _tqdm_notebook, tqdm_pandas
tic = time.time()

In [50]:
#############################user#################################

In [51]:
with open('pkl/user_feat.pkl','rb') as file:
    user=pickle.load(file)

In [52]:
user.head()

,uid,gender,freq,uf_b1,uf_b2,uf_b3,uf_b4,uf_b5,uf_c1,uf_c2,uf_c3,uf_c4,uf_c5,score,follow_topic,inter_topic,follow_topic_vector,inter_topic_vector,topic_vector_0,topic_vector_1,topic_vector_2,topic_vector_3,topic_vector_4,topic_vector_5,topic_vector_6,topic_vector_7,topic_vector_8,topic_vector_9,topic_vector_10,topic_vector_11,topic_vector_12,topic_vector_13,topic_vector_14,topic_vector_15,topic_vector_16,topic_vector_17,topic_vector_18,topic_vector_19,topic_vector_20,topic_vector_21,topic_vector_22,topic_vector_23,topic_vector_24,topic_vector_25,topic_vector_26,topic_vector_27,topic_vector_28,topic_vector_29,topic_vector_30,topic_vector_31,topic_vector_32,topic_vector_33,topic_vector_34,topic_vector_35,topic_vector_36,topic_vector_37,topic_vector_38,topic_vector_39,topic_vector_40,topic_vector_41,topic_vector_42,topic_vector_43,topic_vector_44,topic_vector_45,topic_vector_46,topic_vector_47,topic_vector_48,topic_vector_49,topic_vector_50,topic_vector_51,topic_vector_52,topic_vector_53,topic_vector_54,topic_vector_55,topic_vector_56,topic_vector_57,topic_vector_58,topic_vector_59,topic_vector_60,topic_vector_61,topic_vector_62,topic_vector_63,topic_interestvector_0,topic_interestvector_1,topic_interestvector_2,topic_interestvector_3,topic_interestvector_4,topic_interestvector_5,topic_interestvector_6,topic_interestvector_7,topic_interestvector_8,topic_interestvector_9,topic_interestvector_10,topic_interestvector_11,topic_interestvector_12,topic_interestvector_13,topic_interestvector_14,topic_interestvector_15,topic_interestvector_16,topic_interestvector_17,topic_interestvector_18,topic_interestvector_19,topic_interestvector_20,topic_interestvector_21,topic_interestvector_22,topic_interestvector_23,topic_interestvector_24,topic_interestvector_25,topic_interestvector_26,topic_interestvector_27,topic_interestvector_28,topic_interestvector_29,topic_interestvector_30,topic_interestvector_31,topic_interestvector_32,topic_interestvector_33,topic_interestvector_34,topic_interestvector_35,topic_interestvector_36,topic_interestvector_37,topic_interestvector_38,topic_interestvector_39,topic_interestvector_40,topic_interestvector_41,topic_interestvector_42,topic_interestvector_43,topic_interestvector_44,topic_interestvector_45,topic_interestvector_46,topic_interestvector_47,topic_interestvector_48,topic_interestvector_49,topic_interestvector_50,topic_interestvector_51,topic_interestvector_52,topic_interestvector_53,topic_interestvector_54,topic_interestvector_55,topic_interestvector_56,topic_interestvector_57,topic_interestvector_58,topic_interestvector_59,topic_interestvector_60,topic_interestvector_61,topic_interestvector_62,topic_interestvector_63
0,M1934753188,1,1,0,1,0,1,0,1190,130,396,1438,1,764,[540],"{21107: 1.7915097, 405: 1.6123838, 4436: 1.518...","[6.125305, -1.4180568, -1.3143845, -2.9268239,...","[0.5255145018431924, 2.8507211193389996, 0.571...",6.125305,-1.418057,-1.314385,-2.926824,3.536058,0.420025,-1.636769,1.878827,-3.706468,-5.212338,-1.205067,2.385681,-1.526738,0.208100,-0.889351,-0.722904,-0.480692,3.529726,-0.562937,1.194617,1.075868,-3.757978,-3.580806,-0.236190,2.351880,0.354231,2.900861,1.425630,0.369802,-1.597198,-3.691592,-2.472038,-0.770294,2.222764,-1.149391,4.175584,2.722525,0.052657,-2.161533,-1.821309,1.074579,0.380354,-0.563431,2.680392,0.682462,-1.871369,3.121419,-4.355078,-2.290254,-0.061968,3.690587,-1.880680,-3.261838,1.316867,-1.394143,1.872898,-0.613960,-2.759586,2.899366,-1.668434,-0.407166,2.303850,-0.585271,-1.941906,0.525515,2.850721,0.571936,0.294524,-1.304087,0.033635,2.678171,-2.902922,-0.272031,-1.808802,-0.406976,-1.035584,1.666736,-0.615750,2.778253,0.906166,1.981844,1.062571,-1.443204,8.029341,-1.700677,1.241317,0.911808,-3.194198,-2.200421,1.997505,2.391157,-2.548748,-1.477882,-0.513216,-2.699883,-2.925372,4.291119,-4.140048,1.844560,-1.502163,0.227879,-3.055509,1.070298,3.583576,-1.430313,-0.886752,0.624935,1.428164,-1.562200,2.194569,-0.081492,-5.919505,-0.371944,1.403582,-0.627435,-0

In [53]:
user.shape

(1931654, 146)

In [48]:
columns=['uid']
for i in range(64):
    columns.append('topic_vector_{}'.format(i))
user_topic=user[columns]

KeyError: "['topic_vector_57', 'topic_vector_5', 'topic_vector_10', 'topic_vector_40', 'topic_vector_21', 'topic_vector_31', 'topic_vector_54', 'topic_vector_24', 'topic_vector_28', 'topic_vector_53', 'topic_vector_61', 'topic_vector_4', 'topic_vector_6', 'topic_vector_44', 'topic_vector_46', 'topic_vector_60', 'topic_vector_50', 'topic_vector_37', 'topic_vector_58', 'topic_vector_55', 'topic_vector_20', 'topic_vector_45', 'topic_vector_56', 'topic_vector_2', 'topic_vector_16', 'topic_vector_34', 'topic_vector_13', 'topic_vector_23', 'topic_vector_26', 'topic_vector_3', 'topic_vector_7', 'topic_vector_48', 'topic_vector_9', 'topic_vector_1', 'topic_vector_12', 'topic_vector_19', 'topic_vector_43', 'topic_vector_17', 'topic_vector_0', 'topic_vector_42', 'topic_vector_8', 'topic_vector_39', 'topic_vector_36', 'topic_vector_27', 'topic_vector_11', 'topic_vector_14', 'topic_vector_32', 'topic_vector_18', 'topic_vector_41', 'topic_vector_15', 'topic_vector_51', 'topic_vector_62', 'topic_vec

In [9]:
user_topic_vector=user[columns]

In [10]:
user_topic_vector.head()

,uid,topic_vector_0,topic_vector_1,topic_vector_2,topic_vector_3,topic_vector_4,topic_vector_5,topic_vector_6,topic_vector_7,topic_vector_8,...,topic_vector_54,topic_vector_55,topic_vector_56,topic_vector_57,topic_vector_58,topic_vector_59,topic_vector_60,topic_vector_61,topic_vector_62,topic_vector_63
0,M1934753188,6.125305,-1.418057,-1.314385,-2.926824,3.536058,0.420025,-1.636769,1.878827,-3.706468,...,-1.394143,1.872898,-0.613960,-2.759586,2.899366,-1.668434,-0.407166,2.303850,-0.585271,-1.941906
1,M595924114,0.305752,-0.998052,0.051290,0.340350,-0.632043,-0.867316,1.014756,-1.125777,-0.351588,...,-0.260536,1.628327,-1.281102,0.008155,0.247146,-0.622986,0.591792,-2.081999,1.112626,0.253570
2,M1473482940,0.150803,0.743465,-0.253077,0.604128,-0.587899,0.412107,0.867326,1.339907,0.124655,...,-1.009660,-0.737141,-0.057168,0.408718,-0.406302,0.776222,1.153201,0.124016,-0.289949,-0.838087
3,M578477092,1.460593,-0.339661,0.327213,-1.364520,-0.113531,-0.437634,1.331930,-0.918737,-0.597395,...,-0.396982,2.509422,-1.044082,0.685882,0.227244,0.092655,0.485983,-1.223400,-1.247271,4.146280
4,M1088794709,0.964365,0.315164,0.829562,-1.606779,0.960933,-0.100294,1.576131,1.015088,-0.093659,...,-2.807216,1.072611,-1.802078,-0.247957,0.118991,2.566538,0.323431,-1.252382,-2.061385,0.641430


In [11]:
user_topic_vector.shape

(1931654, 65)

In [8]:
columns=['uid']
for i in range(64):
    columns.append('topic_interestvector_{}'.format(i))
user_intertopic_vector=user[columns]

In [9]:
user_intertopic_vector=user[columns]

In [10]:
user_intertopic_vector.head()

,uid,topic_interestvector_0,topic_interestvector_1,topic_interestvector_2,topic_interestvector_3,topic_interestvector_4,topic_interestvector_5,topic_interestvector_6,topic_interestvector_7,topic_interestvector_8,...,topic_interestvector_54,topic_interestvector_55,topic_interestvector_56,topic_interestvector_57,topic_interestvector_58,topic_interestvector_59,topic_interestvector_60,topic_interestvector_61,topic_interestvector_62,topic_interestvector_63
0,M1934753188,0.525515,2.850721,0.571936,0.294524,-1.304087,0.033635,2.678171,-2.902922,-0.272031,...,-1.312053,1.438943,0.682684,1.152938,-0.531675,2.659670,3.196776,-4.109969,0.735074,3.004685
1,M595924114,-1.994769,1.338741,-0.322591,-0.338373,2.041610,-0.941478,-1.055003,-3.572848,1.843430,...,0.039199,1.783575,0.664132,2.469747,-0.719207,0.986859,0.217027,-0.482656,0.891986,0.583981
2,M1473482940,-0.258244,0.616728,-0.075800,-1.096214,-0.302267,-0.796212,1.036198,-0.798215,-0.281409,...,-0.040453,3.731454,1.367342,-0.495246,-0.922215,1.998748,-1.344386,-0.210083,2.575945,1.354626
3,M578477092,1.490886,1.443867,1.179747,-1.984802,0.777951,0.946910,4.015217,-1.342449,-1.870478,...,1.601662,2.310372,-1.333152,-7.643324,-2.336455,1.493872,-0.043261,-0.103273,-3.988521,1.952898
4,M1088794709,1.587930,-1.145353,-0.704734,-0.695071,-1.801123,0.700587,2.046240,-1.459716,-1.477486,...,-1.788077,0.773613,-2.791936,-1.405117,-0.113664,1.477649,-0.371154,1.523348,2.211292,3.806222


In [ ]:
###########################  添加 keyword_vector_64

In [15]:
with open('pkl/user_keyword_feat.pkl','rb') as file:
    user_keyword=pickle.load(file)

In [16]:
user_keyword.head()

,uid,keyword_vector_0,keyword_vector_1,keyword_vector_2,keyword_vector_3,keyword_vector_4,keyword_vector_5,keyword_vector_6,keyword_vector_7,keyword_vector_8,...,keyword_vector_54,keyword_vector_55,keyword_vector_56,keyword_vector_57,keyword_vector_58,keyword_vector_59,keyword_vector_60,keyword_vector_61,keyword_vector_62,keyword_vector_63
0,M1934753188,0.125612,-0.572689,-0.144789,-0.052494,-0.036887,-0.287028,0.021398,-0.306155,-0.252065,...,0.182745,0.531644,-0.277505,-0.337594,-0.470887,-0.333244,-0.105863,0.155339,-0.305796,0.068118
1,M595924114,0.125612,-0.572689,-0.144789,-0.052494,-0.036887,-0.287028,0.021398,-0.306155,-0.252065,...,0.182745,0.531644,-0.277505,-0.337594,-0.470887,-0.333244,-0.105863,0.155339,-0.305796,0.068118
2,M1473482940,0.125612,-0.572689,-0.144789,-0.052494,-0.036887,-0.287028,0.021398,-0.306155,-0.252065,...,0.182745,0.531644,-0.277505,-0.337594,-0.470887,-0.333244,-0.105863,0.155339,-0.305796,0.068118
3,M578477092,0.125612,-0.572689,-0.144789,-0.052494,-0.036887,-0.287028,0.021398,-0.306155,-0.252065,...,0.182745,0.531644,-0.277505,-0.337594,-0.470887,-0.333244,-0.105863,0.155339,-0.305796,0.068118
4,M1088794709,0.125612,-0.572689,-0.144789,-0.052494,-0.036887,-0.287028,0.021398,-0.306155,-0.252065,...,0.182745,0.531644,-0.277505,-0.337594,-0.470887,-0.333244,-0.105863,0.155339,-0.305796,0.068118


In [12]:
#############################ques#################################

In [3]:
with open('pkl/ques_feat.pkl','rb') as file:
    ques=pickle.load(file)

In [14]:
ques.shape

(1829900, 73)

In [15]:
columns=['qid']
for i in range(64):
    columns.append('questopic_vector_{}'.format(i))
ques_topic=ques[columns]

In [16]:
ques_topic.head()

,qid,questopic_vector_0,questopic_vector_1,questopic_vector_2,questopic_vector_3,questopic_vector_4,questopic_vector_5,questopic_vector_6,questopic_vector_7,questopic_vector_8,...,questopic_vector_54,questopic_vector_55,questopic_vector_56,questopic_vector_57,questopic_vector_58,questopic_vector_59,questopic_vector_60,questopic_vector_61,questopic_vector_62,questopic_vector_63
0,Q2234111670,-1.121143,2.920241,-0.097328,-2.640352,1.895282,-1.846640,-6.146694,-4.181572,-0.034120,...,-2.036547,4.687829,-1.446055,-6.856635,3.313601,5.769560,-4.032069,-0.049398,3.134176,-3.258174
1,Q760329790,-2.680529,-5.466671,-3.977088,-1.476769,0.415902,2.088326,-0.741878,1.256335,2.455551,...,-0.914641,-2.041675,-2.618623,2.232831,-0.208937,-1.164138,-1.395941,-3.594548,-1.831595,-0.726349
2,Q741313548,1.167998,-0.990261,2.861458,2.517082,-0.629097,0.963800,3.996967,-2.698148,1.657959,...,-5.092326,7.119682,2.863095,-5.552940,1.790051,3.361393,-0.737555,-2.759971,3.144501,-1.107774
3,Q3481466230,-1.445572,-1.401717,1.141537,3.502522,-0.491603,-1.247067,1.491883,-0.816897,-2.198769,...,0.938385,-1.170631,0.593306,0.802875,1.326276,0.015383,0.098349,0.474371,-0.240829,-2.683370
4,Q3966197028,1.281498,1.846888,-2.340193,2.048831,-0.290757,-0.484098,-3.582294,-3.108796,-2.256867,...,-2.516062,4.341034,2.401198,-3.433695,1.377361,3.901017,0.714511,-0.669386,-1.145614,1.592404


In [17]:
ques_topic.shape

(1829900, 65)

In [2]:
with open('pkl/ques_feat.pkl','rb') as file:
    ques=pickle.load(file)

In [3]:
ques.shape

(1829900, 203)

In [4]:
columns=['qid']
for i in range(64):
    columns.append('title_w_vector_{}'.format(i))
ques_w_topic=ques[columns]

In [4]:
with open('pkl/ques_feat.pkl','rb') as file:
    ques=pickle.load(file)

In [31]:
ques.shape

(1829900, 203)

In [32]:
columns=['qid']
for i in range(64):
    columns.append('desc_w_vector_{}'.format(i))
ques_w_desc=ques[columns]

In [33]:
ques_w_desc.head()

,qid,desc_w_vector_0,desc_w_vector_1,desc_w_vector_2,desc_w_vector_3,desc_w_vector_4,desc_w_vector_5,desc_w_vector_6,desc_w_vector_7,desc_w_vector_8,...,desc_w_vector_54,desc_w_vector_55,desc_w_vector_56,desc_w_vector_57,desc_w_vector_58,desc_w_vector_59,desc_w_vector_60,desc_w_vector_61,desc_w_vector_62,desc_w_vector_63
0,Q2234111670,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Q760329790,0.040301,-1.361032,-3.941931,-0.051739,0.661915,-0.434258,-0.942627,0.678512,1.432944,...,-1.087839,-0.831629,-2.291951,-1.371644,-1.285822,-0.293933,-0.690330,-3.132176,-1.554128,-0.275067
2,Q741313548,4.495964,1.582831,0.090314,-4.173879,-1.967118,-2.244121,2.730923,-0.682377,-0.147501,...,-1.446025,3.608161,-0.222802,1.530861,-6.145702,-0.830513,7.076328,-1.339278,0.131427,1.022736
3,Q3481466230,1.580925,-1.288990,-0.065777,0.245641,-0.018409,-1.757964,-0.193851,-1.772536,0.312569,...,-0.697504,1.204814,-0.485135,1.045263,-0.929139,-0.488634,-0.944420,-0.926641,-1.725846,-0.682033
4,Q3966197028,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [5]:
ques.shape

(1829900, 333)

In [6]:
columns=['qid']
for i in range(64):
    columns.append('desc_sw_vector_{}'.format(i))
ques_sw_desc=ques[columns]

In [7]:
columns=['qid']
for i in range(64):
    columns.append('title_sw_vector_{}'.format(i))
ques_sw_title=ques[columns]

In [ ]:
#######################################################################

In [18]:
#############################合并################################# user_topic_vector    ques_topic

In [11]:
with open('pkl/data.pkl','rb') as file:
    data_a=pickle.load(file)
data_a.shape

(3735352, 130)

In [21]:
import pandas as pd
import numpy as np
data_a=pd.merge(data_a,user_topic_vector,how='left',left_on='uid',right_on='uid')

In [22]:
data_a.head()

,day,freq,gender,hour,label,q_ans_count,q_diff_qa_days_max,q_diff_qa_days_mean,q_diff_qa_days_sum,q_has_img_max,...,topic_vector_54,topic_vector_55,topic_vector_56,topic_vector_57,topic_vector_58,topic_vector_59,topic_vector_60,topic_vector_61,topic_vector_62,topic_vector_63
0,3865,5,3,22,0.0,NaN,NaN,NaN,NaN,NaN,...,-0.072205,-0.668949,0.046458,-0.970009,-0.079433,-0.164610,0.903557,-0.196221,0.657812,-0.945632
1,3862,5,3,15,0.0,NaN,NaN,NaN,NaN,NaN,...,-1.236460,1.129174,-0.149403,-1.873849,-0.426482,0.065988,0.972086,-0.208866,-0.584990,1.072633
2,3867,2,3,4,0.0,32.0,13.0,9.531250,305.0,1.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,3861,1,2,20,0.0,3.0,234.0,222.333328,667.0,1.0,...,-3.902392,-3.278000,-1.738747,-0.245467,-0.139347,-1.880228,-0.834830,-0.234049,3.734015,0.998140
4,3862,5,1,8,1.0,NaN,NaN,NaN,NaN,NaN,...,-1.201370,-2.053574,-2.468833,-0.301784,0.536084,0.116829,0.919416,2.663238,-0.126760,2.407444


In [23]:
data_a.shape

(3735352, 194)

In [24]:
data_a=pd.merge(data_a,ques_topic,how='left',left_on='qid',right_on='qid')

In [25]:
data_a.head()

,day,freq,gender,hour,label,q_ans_count,q_diff_qa_days_max,q_diff_qa_days_mean,q_diff_qa_days_sum,q_has_img_max,...,questopic_vector_54,questopic_vector_55,questopic_vector_56,questopic_vector_57,questopic_vector_58,questopic_vector_59,questopic_vector_60,questopic_vector_61,questopic_vector_62,questopic_vector_63
0,3865,5,3,22,0.0,NaN,NaN,NaN,NaN,NaN,...,-2.043432,-2.175881,0.750043,0.655443,0.246836,0.861305,1.544460,-1.576067,0.632768,-0.542106
1,3862,5,3,15,0.0,NaN,NaN,NaN,NaN,NaN,...,-2.035546,-1.756996,0.188272,0.447238,-0.936821,-0.933798,-1.079732,0.068953,-0.018498,-1.272667
2,3867,2,3,4,0.0,32.0,13.0,9.531250,305.0,1.0,...,-4.387553,0.412341,0.167728,2.626560,1.622908,0.621534,-1.488130,1.412777,-0.184961,0.633811
3,3861,1,2,20,0.0,3.0,234.0,222.333328,667.0,1.0,...,-2.463593,1.410340,-0.434783,2.733571,0.938118,0.804930,0.498411,0.167077,-0.486622,1.150244
4,3862,5,1,8,1.0,NaN,NaN,NaN,NaN,NaN,...,-0.563142,-2.626269,-1.638979,-1.395928,-2.013623,0.599056,-0.845648,5.638884,-1.799999,0.438575


In [26]:
data_a.shape

(3735352, 258)

In [27]:
#保存处理好的data_a
with open('pkl/data_vecor.pkl','wb') as file:
    pickle.dump(data_a,file)

In [ ]:
#######################################################添加 user_intertopic_vector

In [12]:
with open('pkl/data_vecor.pkl','rb') as file:
    data_a=pickle.load(file)

In [13]:
data_a.head()

,day,freq,gender,hour,label,q_ans_count,q_diff_qa_days_max,q_diff_qa_days_mean,q_diff_qa_days_sum,q_has_img_max,...,questopic_vector_54,questopic_vector_55,questopic_vector_56,questopic_vector_57,questopic_vector_58,questopic_vector_59,questopic_vector_60,questopic_vector_61,questopic_vector_62,questopic_vector_63
0,3865,5,3,22,0.0,NaN,NaN,NaN,NaN,NaN,...,-2.043432,-2.175881,0.750043,0.655443,0.246836,0.861305,1.544460,-1.576067,0.632768,-0.542106
1,3862,5,3,15,0.0,NaN,NaN,NaN,NaN,NaN,...,-2.035546,-1.756996,0.188272,0.447238,-0.936821,-0.933798,-1.079732,0.068953,-0.018498,-1.272667
2,3867,2,3,4,0.0,32.0,13.0,9.531250,305.0,1.0,...,-4.387553,0.412341,0.167728,2.626560,1.622908,0.621534,-1.488130,1.412777,-0.184961,0.633811
3,3861,1,2,20,0.0,3.0,234.0,222.333328,667.0,1.0,...,-2.463593,1.410340,-0.434783,2.733571,0.938118,0.804930,0.498411,0.167077,-0.486622,1.150244
4,3862,5,1,8,1.0,NaN,NaN,NaN,NaN,NaN,...,-0.563142,-2.626269,-1.638979,-1.395928,-2.013623,0.599056,-0.845648,5.638884,-1.799999,0.438575


In [14]:
data_a=pd.merge(data_a,user_intertopic_vector,how='left',left_on='uid',right_on='uid')

In [15]:
data_a.head()

,day,freq,gender,hour,label,q_ans_count,q_diff_qa_days_max,q_diff_qa_days_mean,q_diff_qa_days_sum,q_has_img_max,...,topic_interestvector_54,topic_interestvector_55,topic_interestvector_56,topic_interestvector_57,topic_interestvector_58,topic_interestvector_59,topic_interestvector_60,topic_interestvector_61,topic_interestvector_62,topic_interestvector_63
0,3865,5,3,22,0.0,NaN,NaN,NaN,NaN,NaN,...,-0.901513,-0.979701,-0.018569,-1.565832,-0.944912,-0.217636,-1.705292,0.837225,1.634138,-0.254388
1,3862,5,3,15,0.0,NaN,NaN,NaN,NaN,NaN,...,0.970896,4.038660,-0.067993,-3.368940,-0.472526,3.809227,-4.371904,-3.018367,-1.754200,-0.566915
2,3867,2,3,4,0.0,32.0,13.0,9.531250,305.0,1.0,...,-0.646524,0.183542,-0.189638,1.122452,0.741160,0.967928,1.682913,-0.072807,-0.862097,0.201554
3,3861,1,2,20,0.0,3.0,234.0,222.333328,667.0,1.0,...,-1.791221,0.471029,-0.573838,0.745743,0.056816,-0.658574,-0.602537,1.081479,-0.157845,0.753416
4,3862,5,1,8,1.0,NaN,NaN,NaN,NaN,NaN,...,-1.043529,0.547037,-0.595820,-1.314307,0.402090,-0.187021,-0.888736,1.195731,1.603195,0.240730


In [16]:
#保存处理好的data_a
with open('pkl/data_vecor.pkl','wb') as file:
    pickle.dump(data_a,file)

In [ ]:
#######################################################################添加 ques_w_desc

In [34]:
with open('pkl/data_vecor.pkl','rb') as file:
    data_a=pickle.load(file)

In [35]:
data_a.head()

,day,freq,gender,hour,label,q_ans_count,q_diff_qa_days_max,q_diff_qa_days_mean,q_diff_qa_days_sum,q_has_img_max,...,topic_interestvector_54,topic_interestvector_55,topic_interestvector_56,topic_interestvector_57,topic_interestvector_58,topic_interestvector_59,topic_interestvector_60,topic_interestvector_61,topic_interestvector_62,topic_interestvector_63
0,3865,5,3,22,0.0,NaN,NaN,NaN,NaN,NaN,...,-0.901513,-0.979701,-0.018569,-1.565832,-0.944912,-0.217636,-1.705292,0.837225,1.634138,-0.254388
1,3862,5,3,15,0.0,NaN,NaN,NaN,NaN,NaN,...,0.970896,4.038660,-0.067993,-3.368940,-0.472526,3.809227,-4.371904,-3.018367,-1.754200,-0.566915
2,3867,2,3,4,0.0,32.0,13.0,9.531250,305.0,1.0,...,-0.646524,0.183542,-0.189638,1.122452,0.741160,0.967928,1.682913,-0.072807,-0.862097,0.201554
3,3861,1,2,20,0.0,3.0,234.0,222.333328,667.0,1.0,...,-1.791221,0.471029,-0.573838,0.745743,0.056816,-0.658574,-0.602537,1.081479,-0.157845,0.753416
4,3862,5,1,8,1.0,NaN,NaN,NaN,NaN,NaN,...,-1.043529,0.547037,-0.595820,-1.314307,0.402090,-0.187021,-0.888736,1.195731,1.603195,0.240730


In [36]:
data_a=pd.merge(data_a,ques_w_desc,how='left',left_on='qid',right_on='qid')

In [37]:
data_a.head()

,day,freq,gender,hour,label,q_ans_count,q_diff_qa_days_max,q_diff_qa_days_mean,q_diff_qa_days_sum,q_has_img_max,...,desc_w_vector_54,desc_w_vector_55,desc_w_vector_56,desc_w_vector_57,desc_w_vector_58,desc_w_vector_59,desc_w_vector_60,desc_w_vector_61,desc_w_vector_62,desc_w_vector_63
0,3865,5,3,22,0.0,NaN,NaN,NaN,NaN,NaN,...,-1.435775,-0.610352,0.107990,-0.818876,-1.668678,-0.055059,-0.311293,-0.185245,-0.630949,0.289849
1,3862,5,3,15,0.0,NaN,NaN,NaN,NaN,NaN,...,-1.633592,1.873400,-0.630030,-1.459686,-0.723744,-0.852720,1.310311,-1.088119,-0.978282,1.109435
2,3867,2,3,4,0.0,32.0,13.0,9.531250,305.0,1.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,3861,1,2,20,0.0,3.0,234.0,222.333328,667.0,1.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,3862,5,1,8,1.0,NaN,NaN,NaN,NaN,NaN,...,-0.668929,-2.399537,-1.432307,-1.929978,-0.871140,0.583352,-0.480967,0.827088,3.129458,5.162716


In [38]:
#保存处理好的data_a
with open('pkl/data_vecor.pkl','wb') as file:
    pickle.dump(data_a,file)

In [ ]:
#######################################################################添加 ques_w_topic

In [5]:
with open('pkl/data_vecor.pkl','rb') as file:
    data_a=pickle.load(file)

In [6]:
data_a.head()

,day,freq,gender,hour,label,q_ans_count,q_diff_qa_days_max,q_diff_qa_days_mean,q_diff_qa_days_sum,q_has_img_max,...,desc_w_vector_54,desc_w_vector_55,desc_w_vector_56,desc_w_vector_57,desc_w_vector_58,desc_w_vector_59,desc_w_vector_60,desc_w_vector_61,desc_w_vector_62,desc_w_vector_63
0,3865,5,3,22,0.0,NaN,NaN,NaN,NaN,NaN,...,-1.435775,-0.610352,0.107990,-0.818876,-1.668678,-0.055059,-0.311293,-0.185245,-0.630949,0.289849
1,3862,5,3,15,0.0,NaN,NaN,NaN,NaN,NaN,...,-1.633592,1.873400,-0.630030,-1.459686,-0.723744,-0.852720,1.310311,-1.088119,-0.978282,1.109435
2,3867,2,3,4,0.0,32.0,13.0,9.531250,305.0,1.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,3861,1,2,20,0.0,3.0,234.0,222.333328,667.0,1.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,3862,5,1,8,1.0,NaN,NaN,NaN,NaN,NaN,...,-0.668929,-2.399537,-1.432307,-1.929978,-0.871140,0.583352,-0.480967,0.827088,3.129458,5.162716


In [7]:
data_a=pd.merge(data_a,ques_w_topic,how='left',left_on='qid',right_on='qid')

In [8]:
data_a.head()

,day,freq,gender,hour,label,q_ans_count,q_diff_qa_days_max,q_diff_qa_days_mean,q_diff_qa_days_sum,q_has_img_max,...,title_w_vector_54,title_w_vector_55,title_w_vector_56,title_w_vector_57,title_w_vector_58,title_w_vector_59,title_w_vector_60,title_w_vector_61,title_w_vector_62,title_w_vector_63
0,3865,5,3,22,0.0,NaN,NaN,NaN,NaN,NaN,...,-2.138695,-1.196315,-0.848112,-1.590491,-2.203341,0.516174,2.612662,-0.020300,-2.081824,1.556658
1,3862,5,3,15,0.0,NaN,NaN,NaN,NaN,NaN,...,-4.378286,1.013507,-0.027744,-0.770079,-0.393093,1.921262,1.203239,0.282538,-1.570819,0.065542
2,3867,2,3,4,0.0,32.0,13.0,9.531250,305.0,1.0,...,-3.028660,1.807451,-0.408985,3.099575,1.908134,1.458719,-1.282697,2.003137,0.697350,-0.145008
3,3861,1,2,20,0.0,3.0,234.0,222.333328,667.0,1.0,...,-5.627925,0.691021,4.544005,1.343829,5.936729,-0.682514,2.679075,1.316467,-2.246121,2.465380
4,3862,5,1,8,1.0,NaN,NaN,NaN,NaN,NaN,...,-5.173309,-1.509288,1.097261,-0.891154,-3.662337,1.453739,-4.452105,6.391893,-2.334040,2.314226


In [9]:
#保存处理好的data_a
with open('pkl/data_vecor.pkl','wb') as file:
    pickle.dump(data_a,file)

In [ ]:
#######################################################################添加 desc_sw_vector

In [8]:
with open('pkl/data_vecor.pkl','rb') as file:
    data_a=pickle.load(file)

In [9]:
data_a.shape

(3735352, 450)

In [10]:
data_a=pd.merge(data_a,ques_sw_desc,how='left',left_on='qid',right_on='qid')

In [11]:
data_a.shape

(3735352, 514)

In [ ]:
#######################################################################添加 title_sw_vector

In [12]:
data_a=pd.merge(data_a,ques_sw_title,how='left',left_on='qid',right_on='qid')

In [13]:
data_a.shape

(3735352, 578)

In [14]:
#保存处理好的data_a
with open('pkl/data_vecor.pkl','wb') as file:
    pickle.dump(data_a,file)

In [ ]:
#######################################################################添加 user_keyword

In [17]:
with open('pkl/data_vecor.pkl','rb') as file:
    data_a=pickle.load(file)

In [18]:
data_a=pd.merge(data_a,user_keyword,how='left',left_on='uid',right_on='uid')

In [19]:
data_a.shape

(3735352, 642)

In [20]:
#保存处理好的data_a
with open('pkl/data_vecor.pkl','wb') as file:
    pickle.dump(data_a,file)

In [ ]:
########################################################################################################################################
#############################################模型######################################################################################

In [17]:
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from lightgbm import LGBMClassifier
import logging
import pickle

In [18]:
with open('pkl/data_vecor.pkl','rb') as file:
    data_a=pickle.load(file)

In [19]:
data_a.head()

,day,freq,gender,hour,label,q_ans_count,q_diff_qa_days_max,q_diff_qa_days_mean,q_diff_qa_days_sum,q_has_img_max,...,topic_interestvector_54,topic_interestvector_55,topic_interestvector_56,topic_interestvector_57,topic_interestvector_58,topic_interestvector_59,topic_interestvector_60,topic_interestvector_61,topic_interestvector_62,topic_interestvector_63
0,3865,5,3,22,0.0,NaN,NaN,NaN,NaN,NaN,...,-0.901513,-0.979701,-0.018569,-1.565832,-0.944912,-0.217636,-1.705292,0.837225,1.634138,-0.254388
1,3862,5,3,15,0.0,NaN,NaN,NaN,NaN,NaN,...,0.970896,4.038660,-0.067993,-3.368940,-0.472526,3.809227,-4.371904,-3.018367,-1.754200,-0.566915
2,3867,2,3,4,0.0,32.0,13.0,9.531250,305.0,1.0,...,-0.646524,0.183542,-0.189638,1.122452,0.741160,0.967928,1.682913,-0.072807,-0.862097,0.201554
3,3861,1,2,20,0.0,3.0,234.0,222.333328,667.0,1.0,...,-1.791221,0.471029,-0.573838,0.745743,0.056816,-0.658574,-0.602537,1.081479,-0.157845,0.753416
4,3862,5,1,8,1.0,NaN,NaN,NaN,NaN,NaN,...,-1.043529,0.547037,-0.595820,-1.314307,0.402090,-0.187021,-0.888736,1.195731,1.603195,0.240730


In [3]:
#######################################################################

In [ ]:
#######################################################################

In [20]:
feature_cols = [x for x in data_a.columns if x not in ('label', 'uid', 'qid', 'dt', 'day')]
# target编码

In [21]:
# train_label = train[(train['day'] > train_label_feature_end)]
# print(len(train_label))
train_label=2593669

In [22]:
X_train_all = data_a.iloc[:train_label][feature_cols]
y_train_all = data_a.iloc[:train_label]['label']
test = data_a.iloc[train_label:]

logging.info("train shape %s, test shape %s", X_train_all.shape, test.shape)


[2019-12-06 15:43:19,980] INFO in <ipython-input-22-974f0c82ab38>: train shape (2593669, 318), test shape (1141683, 322)


In [23]:
print(X_train_all.shape)
print(test.shape)

(2593669, 318)
(1141683, 322)


In [24]:

model_lgb = LGBMClassifier(boosting_type='gbdt', num_leaves=64, learning_rate=0.01, n_estimators=2000,
                           max_bin=425, subsample_for_bin=50000, objective='binary', min_split_gain=0,
                           min_child_weight=5, min_child_samples=10, subsample=0.8, subsample_freq=1,
                           colsample_bytree=1, reg_alpha=3, reg_lambda=5, seed=1000, n_jobs=-1, silent=True)

In [25]:
fold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
for index, (train_idx, val_idx) in enumerate(fold.split(X=X_train_all, y=y_train_all)):
    X_train, X_val, y_train, y_val = X_train_all.iloc[train_idx][feature_cols], X_train_all.iloc[val_idx][feature_cols], \
                                     y_train_all.iloc[train_idx], \
                                     y_train_all.iloc[val_idx]
    model_lgb.fit(X_train, y_train,
                  eval_metric=['logloss', 'auc'],
                  eval_set=[(X_val, y_val)],
                  early_stopping_rounds=10)
    

[1]	valid_0's auc: 0.777479	valid_0's binary_logloss: 0.438113
Training until validation scores don't improve for 10 rounds
[2]	valid_0's auc: 0.779721	valid_0's binary_logloss: 0.436586
[3]	valid_0's auc: 0.782432	valid_0's binary_logloss: 0.435107
[4]	valid_0's auc: 0.782633	valid_0's binary_logloss: 0.433669
[5]	valid_0's auc: 0.786831	valid_0's binary_logloss: 0.432266
[6]	valid_0's auc: 0.788174	valid_0's binary_logloss: 0.430894
[7]	valid_0's auc: 0.789807	valid_0's binary_logloss: 0.429552
[8]	valid_0's auc: 0.789859	valid_0's binary_logloss: 0.428263
[9]	valid_0's auc: 0.790316	valid_0's binary_logloss: 0.426998
[10]	valid_0's auc: 0.790889	valid_0's binary_logloss: 0.425749
[11]	valid_0's auc: 0.79088	valid_0's binary_logloss: 0.424542
[12]	valid_0's auc: 0.790772	valid_0's binary_logloss: 0.423374
[13]	valid_0's auc: 0.791066	valid_0's binary_logloss: 0.422216
[14]	valid_0's auc: 0.791845	valid_0's binary_logloss: 0.421067
[15]	valid_0's auc: 0.792428	valid_0's binary_logloss

[129]	valid_0's auc: 0.812979	valid_0's binary_logloss: 0.362992
[130]	valid_0's auc: 0.813096	valid_0's binary_logloss: 0.362783
[131]	valid_0's auc: 0.813213	valid_0's binary_logloss: 0.362572
[132]	valid_0's auc: 0.81335	valid_0's binary_logloss: 0.36236
[133]	valid_0's auc: 0.813457	valid_0's binary_logloss: 0.362146
[134]	valid_0's auc: 0.813581	valid_0's binary_logloss: 0.361928
[135]	valid_0's auc: 0.813706	valid_0's binary_logloss: 0.361715
[136]	valid_0's auc: 0.813819	valid_0's binary_logloss: 0.361517
[137]	valid_0's auc: 0.813936	valid_0's binary_logloss: 0.361297
[138]	valid_0's auc: 0.814055	valid_0's binary_logloss: 0.361094
[139]	valid_0's auc: 0.814145	valid_0's binary_logloss: 0.360891
[140]	valid_0's auc: 0.814257	valid_0's binary_logloss: 0.360678
[141]	valid_0's auc: 0.814358	valid_0's binary_logloss: 0.360482
[142]	valid_0's auc: 0.814448	valid_0's binary_logloss: 0.360293
[143]	valid_0's auc: 0.814592	valid_0's binary_logloss: 0.360091
[144]	valid_0's auc: 0.8146

[256]	valid_0's auc: 0.825073	valid_0's binary_logloss: 0.345469
[257]	valid_0's auc: 0.825145	valid_0's binary_logloss: 0.345382
[258]	valid_0's auc: 0.825206	valid_0's binary_logloss: 0.345302
[259]	valid_0's auc: 0.825279	valid_0's binary_logloss: 0.345209
[260]	valid_0's auc: 0.825336	valid_0's binary_logloss: 0.345137
[261]	valid_0's auc: 0.825418	valid_0's binary_logloss: 0.345032
[262]	valid_0's auc: 0.825494	valid_0's binary_logloss: 0.344945
[263]	valid_0's auc: 0.82556	valid_0's binary_logloss: 0.344866
[264]	valid_0's auc: 0.825623	valid_0's binary_logloss: 0.344783
[265]	valid_0's auc: 0.825684	valid_0's binary_logloss: 0.344707
[266]	valid_0's auc: 0.82574	valid_0's binary_logloss: 0.344626
[267]	valid_0's auc: 0.825791	valid_0's binary_logloss: 0.344558
[268]	valid_0's auc: 0.825841	valid_0's binary_logloss: 0.344494
[269]	valid_0's auc: 0.825898	valid_0's binary_logloss: 0.34441
[270]	valid_0's auc: 0.82596	valid_0's binary_logloss: 0.344325
[271]	valid_0's auc: 0.826015

[383]	valid_0's auc: 0.831579	valid_0's binary_logloss: 0.337679
[384]	valid_0's auc: 0.831609	valid_0's binary_logloss: 0.337642
[385]	valid_0's auc: 0.831645	valid_0's binary_logloss: 0.337603
[386]	valid_0's auc: 0.831681	valid_0's binary_logloss: 0.33756
[387]	valid_0's auc: 0.831712	valid_0's binary_logloss: 0.337526
[388]	valid_0's auc: 0.831751	valid_0's binary_logloss: 0.337484
[389]	valid_0's auc: 0.831786	valid_0's binary_logloss: 0.337447
[390]	valid_0's auc: 0.831822	valid_0's binary_logloss: 0.3374
[391]	valid_0's auc: 0.831855	valid_0's binary_logloss: 0.337366
[392]	valid_0's auc: 0.831894	valid_0's binary_logloss: 0.337326
[393]	valid_0's auc: 0.831931	valid_0's binary_logloss: 0.337289
[394]	valid_0's auc: 0.831961	valid_0's binary_logloss: 0.337248
[395]	valid_0's auc: 0.831998	valid_0's binary_logloss: 0.33721
[396]	valid_0's auc: 0.832033	valid_0's binary_logloss: 0.337171
[397]	valid_0's auc: 0.83207	valid_0's binary_logloss: 0.337131
[398]	valid_0's auc: 0.832119	

[510]	valid_0's auc: 0.835551	valid_0's binary_logloss: 0.333334
[511]	valid_0's auc: 0.835575	valid_0's binary_logloss: 0.333308
[512]	valid_0's auc: 0.835597	valid_0's binary_logloss: 0.33328
[513]	valid_0's auc: 0.835616	valid_0's binary_logloss: 0.333258
[514]	valid_0's auc: 0.835637	valid_0's binary_logloss: 0.333238
[515]	valid_0's auc: 0.835662	valid_0's binary_logloss: 0.333213
[516]	valid_0's auc: 0.835702	valid_0's binary_logloss: 0.333173
[517]	valid_0's auc: 0.835728	valid_0's binary_logloss: 0.333145
[518]	valid_0's auc: 0.835748	valid_0's binary_logloss: 0.333124
[519]	valid_0's auc: 0.835768	valid_0's binary_logloss: 0.333102
[520]	valid_0's auc: 0.835787	valid_0's binary_logloss: 0.333081
[521]	valid_0's auc: 0.835811	valid_0's binary_logloss: 0.333056
[522]	valid_0's auc: 0.83584	valid_0's binary_logloss: 0.333025
[523]	valid_0's auc: 0.83587	valid_0's binary_logloss: 0.333002
[524]	valid_0's auc: 0.835891	valid_0's binary_logloss: 0.332975
[525]	valid_0's auc: 0.83591

...

[1451]	valid_0's auc: 0.848506	valid_0's binary_logloss: 0.321247
[1452]	valid_0's auc: 0.848511	valid_0's binary_logloss: 0.321241
[1453]	valid_0's auc: 0.848516	valid_0's binary_logloss: 0.321236
[1454]	valid_0's auc: 0.84852	valid_0's binary_logloss: 0.321234
[1455]	valid_0's auc: 0.848525	valid_0's binary_logloss: 0.321229
[1456]	valid_0's auc: 0.848533	valid_0's binary_logloss: 0.321222
[1457]	valid_0's auc: 0.848538	valid_0's binary_logloss: 0.321217
[1458]	valid_0's auc: 0.848543	valid_0's binary_logloss: 0.321214
[1459]	valid_0's auc: 0.848551	valid_0's binary_logloss: 0.321207
[1460]	valid_0's auc: 0.848559	valid_0's binary_logloss: 0.321201
[1461]	valid_0's auc: 0.848562	valid_0's binary_logloss: 0.321198
[1462]	valid_0's auc: 0.848568	valid_0's binary_logloss: 0.321191
[1463]	valid_0's auc: 0.848571	valid_0's binary_logloss: 0.321189
[1464]	valid_0's auc: 0.848575	valid_0's binary_logloss: 0.321186
[1465]	valid_0's auc: 0.848583	valid_0's binary_logloss: 0.321179
[1466]	vali

[1576]	valid_0's auc: 0.849215	valid_0's binary_logloss: 0.320598
[1577]	valid_0's auc: 0.849222	valid_0's binary_logloss: 0.320592
[1578]	valid_0's auc: 0.849232	valid_0's binary_logloss: 0.320583
[1579]	valid_0's auc: 0.849237	valid_0's binary_logloss: 0.320579
[1580]	valid_0's auc: 0.849242	valid_0's binary_logloss: 0.320574
[1581]	valid_0's auc: 0.84925	valid_0's binary_logloss: 0.320568
[1582]	valid_0's auc: 0.849256	valid_0's binary_logloss: 0.320563
[1583]	valid_0's auc: 0.84926	valid_0's binary_logloss: 0.320559
[1584]	valid_0's auc: 0.849267	valid_0's binary_logloss: 0.320553
[1585]	valid_0's auc: 0.849272	valid_0's binary_logloss: 0.320549
[1586]	valid_0's auc: 0.849277	valid_0's binary_logloss: 0.320544
[1587]	valid_0's auc: 0.849281	valid_0's binary_logloss: 0.320541
[1588]	valid_0's auc: 0.849286	valid_0's binary_logloss: 0.320537
[1589]	valid_0's auc: 0.849288	valid_0's binary_logloss: 0.320534
[1590]	valid_0's auc: 0.849295	valid_0's binary_logloss: 0.320529
[1591]	valid

[1701]	valid_0's auc: 0.849864	valid_0's binary_logloss: 0.320016
[1702]	valid_0's auc: 0.849869	valid_0's binary_logloss: 0.320011
[1703]	valid_0's auc: 0.849876	valid_0's binary_logloss: 0.320006
[1704]	valid_0's auc: 0.849879	valid_0's binary_logloss: 0.320003
[1705]	valid_0's auc: 0.849884	valid_0's binary_logloss: 0.319998
[1706]	valid_0's auc: 0.849889	valid_0's binary_logloss: 0.319994
[1707]	valid_0's auc: 0.849893	valid_0's binary_logloss: 0.31999
[1708]	valid_0's auc: 0.849898	valid_0's binary_logloss: 0.319987
[1709]	valid_0's auc: 0.849902	valid_0's binary_logloss: 0.319984
[1710]	valid_0's auc: 0.849904	valid_0's binary_logloss: 0.319981
[1711]	valid_0's auc: 0.849912	valid_0's binary_logloss: 0.319974
[1712]	valid_0's auc: 0.849916	valid_0's binary_logloss: 0.31997
[1713]	valid_0's auc: 0.849922	valid_0's binary_logloss: 0.319965
[1714]	valid_0's auc: 0.849926	valid_0's binary_logloss: 0.319962
[1715]	valid_0's auc: 0.849932	valid_0's binary_logloss: 0.319957
[1716]	valid

[1826]	valid_0's auc: 0.85042	valid_0's binary_logloss: 0.319515
[1827]	valid_0's auc: 0.850424	valid_0's binary_logloss: 0.319511
[1828]	valid_0's auc: 0.850426	valid_0's binary_logloss: 0.319509
[1829]	valid_0's auc: 0.850433	valid_0's binary_logloss: 0.319503
[1830]	valid_0's auc: 0.850435	valid_0's binary_logloss: 0.319501
[1831]	valid_0's auc: 0.85044	valid_0's binary_logloss: 0.319498
[1832]	valid_0's auc: 0.850444	valid_0's binary_logloss: 0.319494
[1833]	valid_0's auc: 0.850448	valid_0's binary_logloss: 0.319491
[1834]	valid_0's auc: 0.850453	valid_0's binary_logloss: 0.319486
[1835]	valid_0's auc: 0.850458	valid_0's binary_logloss: 0.319482
[1836]	valid_0's auc: 0.850461	valid_0's binary_logloss: 0.31948
[1837]	valid_0's auc: 0.850466	valid_0's binary_logloss: 0.319475
[1838]	valid_0's auc: 0.85047	valid_0's binary_logloss: 0.319472
[1839]	valid_0's auc: 0.850476	valid_0's binary_logloss: 0.319466
[1840]	valid_0's auc: 0.850482	valid_0's binary_logloss: 0.319461
[1841]	valid_0

[1951]	valid_0's auc: 0.850874	valid_0's binary_logloss: 0.319107
[1952]	valid_0's auc: 0.850876	valid_0's binary_logloss: 0.319105
[1953]	valid_0's auc: 0.850884	valid_0's binary_logloss: 0.319096
[1954]	valid_0's auc: 0.850889	valid_0's binary_logloss: 0.319091
[1955]	valid_0's auc: 0.850893	valid_0's binary_logloss: 0.319088
[1956]	valid_0's auc: 0.850897	valid_0's binary_logloss: 0.319084
[1957]	valid_0's auc: 0.850902	valid_0's binary_logloss: 0.31908
[1958]	valid_0's auc: 0.850904	valid_0's binary_logloss: 0.319079
[1959]	valid_0's auc: 0.850907	valid_0's binary_logloss: 0.319076
[1960]	valid_0's auc: 0.85091	valid_0's binary_logloss: 0.319074
[1961]	valid_0's auc: 0.850911	valid_0's binary_logloss: 0.319072
[1962]	valid_0's auc: 0.850915	valid_0's binary_logloss: 0.319069
[1963]	valid_0's auc: 0.850917	valid_0's binary_logloss: 0.319067
[1964]	valid_0's auc: 0.85092	valid_0's binary_logloss: 0.319064
[1965]	valid_0's auc: 0.850922	valid_0's binary_logloss: 0.319063
[1966]	valid_

In [26]:
sub = pd.read_csv(f'/home/hisense/wyl/data/test.txt', sep='\t', header=None)
sub.columns = ['qid', 'uid', 'dt']
sub['label'] = model_lgb.predict_proba(test[feature_cols])[:, 1]
sub.to_csv('result.txt', index=None, header=None, sep='\t')